<a href="https://colab.research.google.com/github/brunoalbrito/tcc-mackenzie/blob/master/Entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Processamento de Linguagem Natural com finalidade de Encontrar Padrões para FakeNews

Com a utlização da biblioteca NLTK(Natural Language ToolKit), esse notebook tem como objetivo prover formas básicas de processamento de texto para possibilitar a visualização de alguns padrões semânticos e gramaticais encontrados em FakeNews. Como os dicionários com melhor divisão das categorias de palavras foram encontrados em inglês, optou-se por utilizar notícias no mesmo idioma. 

In [0]:
import nltk
import re
import string
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, blankline_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import ne_chunk, word_tokenize
import os
from google.colab import drive
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
import pandas as pd

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')


wlem = WordNetLemmatizer()

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Os métodos abaixo tem como função carregar os arquivos salvos previamente no Google Drive, contendo notícias retiradas da base Liar:
https://github.com/sumeetkr/AwesomeFakeNews

Essa base de dados contém uma série de sites com informações falsas e verdadeiras.

In [0]:
drive.mount('/content/gdrive')
ROOT_PATH = "/content/gdrive/My Drive/8º Semestre/"
packFiles = os.path.join(ROOT_PATH, "TCC II/Noticias/")

def load_data(data_dir):
    file_names = []
    
    label_dir = data_dir
    file_names = [os.path.join(label_dir, i) 
                  for i in os.listdir(label_dir)]
      
    return file_names

def open_files(data_dir):
  news_text = []
  for n in data_dir:
    with open(n, 'r', encoding="utf-8-sig") as f:
      news_text.append(f.read())
  return news_text

news = open_files(load_data(packFiles))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Quantidade de notícias carregadas do Drive
print(len(news))

20


A função ***words_lemmatizer*** primeiramente realizar a tokenização das palavras e ao chamar  ***find_pos***, que é utilizada para detecção de cada categoria em que a palavra se encontra, aproveita o resultado obtido para preencher o dicionário.

In [0]:
def words_lemmatizer(text, encoding='utf8'):
  
  #gerando tokens da palavra
  pos = (ne_chunk(nltk.pos_tag(word_tokenize(text)), binary=False))
  
  #criando dicionario com as tags
  dictionary = {"coordinating conjunction": "", 
                "cardinal digit": "", 
                "determiner": "", 
                "existential there": "", 
                "foreign word": "", 
                "preposition/subordinating": "", 
                "adjective": "",
                "adjective, comparative": "", 
                "adjective, superlative": "", 
                "list marker": "", 
                "modal": "", 
                "noun, singular": "", 
                "noun plural": "", 
                "proper noun, singular": "", 
                "proper noun, plural": "", 
                "predeterminer": "",
                "possessive ending": "", 
                "personal pronoun": "", 
                "possessive pronoun": "",
                "adverb": "", 
                "adverb, comparative": "", 
                "adverb, superlative": "", 
                "particle": "", 
                "to": "", 
                "interjection": "",
                "verb, base form	take": "", 
                "verb, past tense": "", 
                "verb, gerund/present participle": "", 
                "verb, past participle": "", 
                "verb, sing. present": "",
                "verb, 3rd person sing. present": "", 
                "wh-determiner": "", 
                "wh-pronoun": "", 
                "possessive wh-pronoun": "", 
                "wh-abverb": ""}
  
  #criando uma lista vazia
  lemma_words = []
  
  #instancia de WordNetLemmatizer
  wl = WordNetLemmatizer()
    
  #adicionando as palavras no dicionario
  for i in range (0, len(pos)):
    if type(pos[i][0]) is str:
      pos2 = find_pos(pos[i][1])
      dictionary[pos2] += pos[i][0] + " "
    else:
      pos2 = find_pos(pos[i][0][1])
      dictionary[pos2] += pos[i][0][0] + " "
  
  return dictionary


def find_pos(pos):  
  
  #retornando os respectivos valores de cada tag
  if pos.lower() == "cc":
    return 'coordinating conjunction'
  if pos.lower() == "cd":
    return 'cardinal digit'
  if pos.lower() == "dt":
    return 'determiner'
  if pos.lower() == "ex":
    return 'existential there'
  if pos.lower() == "fw":
    return 'foreign word'
  if pos.lower() == "in":
    return 'preposition/subordinating'
  if pos.lower() == "jj":
    return 'adjective'
  if pos.lower() == "jjr":
    return 'adjective, comparative'
  if pos.lower() == "jjs":
    return 'adjective, superlative'
  if pos.lower() == "ls":
    return 'list marker'
  if pos.lower() == "md":
    return 'modal'
  if pos.lower() == "nn":
    return 'noun, singular'
  if pos.lower() == "nns":
    return 'noun plural'
  if pos.lower() == "nnp":
    return 'proper noun, singular'
  if pos.lower() == "nnps":
    return 'proper noun, plural'
  if pos.lower() == "pdt":
    return 'predeterminer'
  if pos.lower() == "pos":
    return 'possessive ending'
  if pos.lower() == "prp":
    return 'personal pronoun'
  if pos.lower() == "prp$":
    return 'possessive pronoun'
  if pos.lower() == "rb":
    return 'adverb'
  if pos.lower() == "rbr":
    return 'adverb, comparative'
  if pos.lower() == "rbs":
    return 'adverb, superlative'
  if pos.lower() == "rp":
    return 'particle'
  if pos.lower() == "to":
    return 'to'
  if pos.lower() == "uh":
    return 'interjection'
  if pos.lower() == "vb":
    return 'verb, base form	take'
  if pos.lower() == "vbd":
    return 'verb, past tense'
  if pos.lower() == "vbg":
    return 'verb, gerund/present participle'
  if pos.lower() == "vbn":
    return 'verb, past participle'
  if pos.lower() == "vbp":
    return 'verb, sing. present'
  if pos.lower() == "vbz":
    return 'verb, 3rd person sing. present'
  if pos.lower() == "wdt":
    return 'wh-determiner'
  if pos.lower() == "wp":
    return 'wh-pronoun'
  if pos.lower() == "wp$":
    return 'possessive wh-pronoun'
  if pos.lower() == "wrb":
    return 'wh-abverb'
  else:
    return pos.lower()


In [0]:
pesos = {"coordinating conjunction": 1, 
                "cardinal digit": 1, 
                "determiner": -1, 
                "existential there": -1, 
                "foreign word": 1, 
                "preposition/subordinating": 1, 
                "adjective": 1,
                "adjective, comparative": -1, 
                "adjective, superlative": -1, 
                "list marker": 1, 
                "modal": -1, 
                "noun, singular": 1, 
                "noun plural": 1, 
                "proper noun, singular": -1, 
                "proper noun, plural": -1, 
                "predeterminer": -1,
                "possessive ending": -1, 
                "personal pronoun": -1, 
                "possessive pronoun": -1,
                "adverb": -1, 
                "adverb, comparative": 1, 
                "adverb, superlative": 1, 
                "particle": 1, 
                "to": 1, 
                "interjection": 1,
                "verb, base form	take": 1, 
                "verb, past tense": -1, 
                "verb, gerund/present participle": -1, 
                "verb, past participle": -1, 
                "verb, sing. present": -1,
                "verb, 3rd person sing. present": -1, 
                "wh-determiner": 1, 
                "wh-pronoun": -1, 
                "possessive wh-pronoun": 1, 
                "wh-abverb": -1}

A notícia passa pelo processo de tratamento, removendo os caracteres que não serão utilizados no processo e assim separando-a em sentenças.

In [0]:
final_dictionary = []

for i in news:
  noticia = re.sub( u"[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ/' ]", '', i)
  texto = words_lemmatizer(noticia)
  final_dictionary.append(texto)

Utilizando o dicionário de sentenças criado anteriormente, realizamos a criação de uma tabela com as palavras, categorias, quantidade de palavras em cada categoria e seus respectivos pesos. 

Cada tabela é salva no diretório em que passamos o caminho. Tendo como nome o número do texto que foi realizado o processo. 

In [0]:
import pandas as pd
from IPython.display import display,HTML

count = 1

for i in final_dictionary:
  display(HTML("<h3>Texto "+str(count)+"</h3>"))
  dicionario_exibicao = {"Palavras": [], "Quantidade": [], "Pesos": []}
  named_keys_dictionary = []
  for key in i.keys():
    if (len(i[key].split(' '))-1) != 0:
      dicionario_exibicao['Palavras'].append( i[key] )
      dicionario_exibicao["Quantidade"].append( len(i[key].split(' '))-1 )
      dicionario_exibicao["Pesos"].append(pesos[key])
      named_keys_dictionary.append(key)
  df = pd.DataFrame(dicionario_exibicao, columns = ["Palavras", "Quantidade", "Pesos"] , index = named_keys_dictionary)
  display(df)
  df.to_csv(r'gdrive/My Drive/8º Semestre/TCC II/Resultados/Classificacao/Texto'+str(count)+'.csv')
  count += 1

,Palavras,Quantidade,Pesos
coordinating conjunction,and and and or,4,1
determiner,the the a a a a an the,8,-1
preposition/subordinating,of In that via that out of for if like,10,1
adjective,real real suspicious proof important real,6,1
"adjective, superlative",latest,1,-1
"noun, singular",father baby girl dad family mom source paterni...,17,1
noun plural,reports shes,2,1
"proper noun, singular",Kylie Jenners Tim Kylie Tim Stormi Travis DNA ...,14,-1
personal pronoun,she him She he,4,-1
possessive pronoun,her his his her its her,6,-1


,Palavras,Quantidade,Pesos
coordinating conjunction,and and and and and and and and and and but bu...,16,1
cardinal digit,2015The three two 2 5foot 49year 180lb 2017 18...,10,1
determiner,a a a a a the the the the no the any the some ...,22,-1
existential there,there,1,-1
preposition/subordinating,by in while on in in of until per in with that...,32,1
adjective,arm CubanAmerican white poor fishing fish fres...,29,1
"adjective, comparative",more,1,-1
"adjective, superlative",most,1,-1
modal,will,1,-1
"noun, singular",bitten pet sharkCesario janitor club baby shar...,49,1


,Palavras,Quantidade,Pesos
coordinating conjunction,and and but and and and and and and and or,11,1
cardinal digit,28yearold 28,2,1
determiner,a the the the the all a the the the the the a,13,-1
preposition/subordinating,after while in after of after from in as by af...,33,1
adjective,critical rare bacteriaAdam severe medical diar...,21,1
modal,can should could,3,-1
"noun, singular",Man disease boyfriend ass man condition fascii...,43,1
noun plural,contracts doctors boyfriends warnings partners...,11,1
"proper noun, singular",A Rader UCSF GHB Benjamin Segura San UCSF,8,-1
possessive ending,'s,1,-1


,Palavras,Quantidade,Pesos
coordinating conjunction,and and and but and and but and and but,10,1
cardinal digit,310 911He 40,3,1
determiner,a the a the the a the the an the this the the ...,27,-1
preposition/subordinating,in after in while from after inside inside fro...,28,1
adjective,letterA postal last stuck narrow container49ye...,32,1
modal,should,1,-1
"noun, singular",stuck mailbox man dwarfism mailbox morning emp...,33,1
noun plural,paramedics minutes hours doctors consequences ...,7,1
"proper noun, singular",Midget US Friday Walter Fayetteville North Mon...,16,-1
personal pronoun,he he I I he I he he he He,10,-1


,Palavras,Quantidade,Pesos
coordinating conjunction,and and and and and and,6,1
cardinal digit,76yearold 911 911 52 870 814 1966Albert 76 70 ...,13,1
determiner,A the a every any the a the the this the this ...,26,-1
preposition/subordinating,because with for in with for at from because w...,24,1
adjective,operatorThe Indian gruesome old reportersHis p...,9,1
modal,occurredIm,1,-1
"noun, singular",Man wife sex decadesFlorida man wife home admi...,42,1
noun plural,times years years wounds,4,1
"proper noun, singular",Fiddler Remington River County Police Departme...,8,-1
predeterminer,all,1,-1


...

,Palavras,Quantidade,Pesos
coordinating conjunction,and and and,3,1
cardinal digit,two 149 one 55000 22000 2018After two 54777 1s...,25,1
determiner,this the the the an all the the The all a the ...,29,-1
existential there,there there there,3,-1
preposition/subordinating,After in in against during in in in that from ...,38,1
adjective,same probable same last central downward same ...,15,1
"adjective, comparative",more better fewer,3,-1
"noun, singular",decline dengue yearSo year period decline deng...,43,1
noun plural,years cases cases years data cases cases cases...,19,1
"proper noun, singular",Brazil New January February That Southeast Zik...,8,-1


,Palavras,Quantidade,Pesos
coordinating conjunction,and and and and and,5,1
cardinal digit,two two 2018 28 seven,5,1
determiner,the the this the the the the the this the the ...,18,-1
preposition/subordinating,from for for on from by that with with with of...,19,1
adjective,Brazilian other new Brazilian open Brazilian i...,11,1
"adjective, comparative",more,1,-1
"adjective, superlative",best,1,-1
modal,might,1,-1
"noun, singular",compensation steel protection government compe...,29,1
noun plural,requests safeguard measures yearsThe tariffs m...,20,1


,Palavras,Quantidade,Pesos
coordinating conjunction,And and But but and and and But and and But an...,13,1
cardinal digit,2019 556 498 608 491 615 580 2018 550 million ...,25,1
determiner,the a this the the a the the some this the The...,21,-1
foreign word,de,1,1
preposition/subordinating,With of in upon in in of with in for in for as...,35,1
adjective,corporate serious last ten permitsThis unlikel...,18,1
"adjective, comparative",more,1,-1
modal,can,1,-1
"noun, singular",street cityOnce time samba year city attendanc...,36,1
noun plural,parades hotels sponsors celebrations parades y...,20,1


,Palavras,Quantidade,Pesos
coordinating conjunction,but and and and and but and and or and and And...,22,1
cardinal digit,27000yearold 27000 13 2014 25 4inchlong two On...,10,1
determiner,tooth a the an a the the the a the the The The...,67,-1
foreign word,seasonsThat diverse,2,1
preposition/subordinating,like on about from that in of like for that of...,71,1
adjective,giant capsuleFossil own past extinct last aliv...,68,1
"adjective, comparative",more lower,2,-1
"adjective, superlative",most most best,3,-1
modal,could could can may might,5,-1
"noun, singular",ground sloth climate time information giant gr...,108,1


,Palavras,Quantidade,Pesos
coordinating conjunction,and and and and and and and and and and and and,12,1
cardinal digit,100 28 42 26000 430 360 45x,7,1
determiner,the this a The the each each a the the the the...,15,-1
existential there,there,1,-1
preposition/subordinating,during in in de during of in with of in from I...,23,1
adjective,facial same facial arrest capable license plat...,17,1
modal,will will will will will will will will will w...,10,-1
"noun, singular",recognition system carnavalThe system year sys...,27,1
noun plural,festivities people warrants records systems of...,17,1
"proper noun, singular",Rio Salvador Bahia ChinaRio Janeiro Salvador C...,16,-1


Realizada a análise de sentimentos de cada notícia e colocando os resultados em outra tabela, com as colunas de "Positivo", "Negativo", "Neutro", "Resultado", sendo salvas também em outro caminho que foi criado. 

In [0]:
count = 1
sentiment = SentimentIntensityAnalyzer()
for i in final_dictionary:
  display(HTML("<h3>Texto "+str(count)+"</h3>"))
  count += 1
  dicionario_exibicao = {"Positivo": [], "Negativo": [], "Neutro": [], "Resultado": []}
  named_keys_dictionary = []
  for key in i.keys():
    if (len(i[key].split(' '))-1) != 0:
      s = sentiment.polarity_scores( i[key])
      dicionario_exibicao['Positivo'].append( s['pos'] )
      dicionario_exibicao["Negativo"].append( s['neg'] )
      dicionario_exibicao["Neutro"].append(s['neu'])
      dicionario_exibicao["Resultado"].append(s['compound'])
      named_keys_dictionary.append(key)
  df = pd.DataFrame(dicionario_exibicao, columns = ["Positivo", "Negativo", "Neutro", "Resultado"] , index = named_keys_dictionary)
  df.to_csv(r'gdrive/My Drive/8º Semestre/TCC II/Resultados/Sentimento/Texto'+str(count)+'.csv')
  display(df)

,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
determiner,0.000,0.000,1.000,0.0000
preposition/subordinating,0.217,0.000,0.783,0.3612
adjective,0.217,0.301,0.482,-0.1779
"adjective, superlative",0.000,0.000,1.000,0.0000
"noun, singular",0.000,0.000,1.000,0.0000
noun plural,0.000,0.000,1.000,0.0000
"proper noun, singular",0.000,0.000,1.000,0.0000
personal pronoun,0.000,0.000,1.000,0.0000
possessive pronoun,0.000,0.000,1.000,0.0000


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.369,0.631,-0.6249
determiner,0.000,0.239,0.761,-0.5267
existential there,0.000,0.000,1.000,0.0000
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.225,0.307,0.468,-0.6908
"adjective, comparative",0.000,0.000,1.000,0.0000
"adjective, superlative",0.000,0.000,1.000,0.0000
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.058,0.110,0.832,-0.4404


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.000,1.000,0.0000
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.131,0.332,0.537,-0.6124
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.021,0.248,0.731,-0.9153
noun plural,0.000,0.375,0.625,-0.6597
"proper noun, singular",0.000,0.000,1.000,0.0000
possessive ending,0.000,0.000,1.000,0.0000


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.104,0.896,-0.2960
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.071,0.272,0.658,-0.8052
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.125,0.131,0.744,-0.0603
noun plural,0.000,0.000,1.000,0.0000
"proper noun, singular",0.000,0.176,0.824,-0.4939
personal pronoun,0.000,0.000,1.000,0.0000


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.106,0.894,-0.3612
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.000,0.000,1.000,0.0000
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.000,0.311,0.689,-0.9493
noun plural,0.000,0.000,1.000,0.0000
"proper noun, singular",0.000,0.000,1.000,0.0000
predeterminer,0.000,0.000,1.000,0.0000


...

,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.000,1.000,0.0000
existential there,0.000,0.000,1.000,0.0000
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.000,0.000,1.000,0.0000
"adjective, comparative",0.615,0.000,0.385,0.4927
"noun, singular",0.052,0.000,0.948,0.3182
noun plural,0.000,0.000,1.000,0.0000
"proper noun, singular",0.000,0.000,1.000,0.0000


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.000,1.000,0.0000
preposition/subordinating,0.122,0.000,0.878,0.3612
adjective,0.206,0.000,0.794,0.3818
"adjective, comparative",0.000,0.000,1.000,0.0000
"adjective, superlative",1.000,0.000,0.000,0.6369
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.113,0.089,0.798,-0.0516
noun plural,0.292,0.000,0.708,0.7184


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.149,0.000,0.851,0.6369
determiner,0.000,0.000,1.000,0.0000
foreign word,0.000,0.000,1.000,0.0000
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.094,0.068,0.838,0.1280
"adjective, comparative",0.000,0.000,1.000,0.0000
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.102,0.157,0.741,-0.5267
noun plural,0.140,0.000,0.860,0.4767


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.000,1.000,0.0000
foreign word,0.000,0.000,1.000,0.0000
preposition/subordinating,0.099,0.000,0.901,0.7579
adjective,0.223,0.046,0.731,0.9231
"adjective, comparative",0.000,0.714,0.286,-0.3597
"adjective, superlative",0.705,0.000,0.295,0.6976
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.028,0.012,0.960,0.4215


,Positivo,Negativo,Neutro,Resultado
coordinating conjunction,0.000,0.000,1.000,0.0000
cardinal digit,0.000,0.000,1.000,0.0000
determiner,0.000,0.000,1.000,0.0000
existential there,0.000,0.000,1.000,0.0000
preposition/subordinating,0.000,0.000,1.000,0.0000
adjective,0.220,0.229,0.551,-0.0516
modal,0.000,0.000,1.000,0.0000
"noun, singular",0.085,0.000,0.915,0.3400
noun plural,0.162,0.000,0.838,0.4767
"proper noun, singular",0.000,0.000,1.000,0.0000
